In [11]:
import numpy as np
from numpy import random
import pandas as pd
import scipy.stats as st
from pathlib import Path
import getInvalid
import PreSW

In [12]:
template=pd.read_csv("Design Study Template.csv", encoding="ISO-8859-1")
df=template.copy()
# print(df)
df.reset_index(inplace=True)  
if "index" in df.columns:
    df.drop(["index"], axis=1, inplace=True)
# print(df)
df.drop(df.index[1], inplace=True)
df.drop(df.columns[0:2], axis=1, inplace=True)

df.columns = df.iloc[0]
# print(df.columns)
df.drop(df.index[0], inplace=True)

#Define specific materials to insert into design templates
steel=df.at[2, " Material"]
aluminum=df.at[3, " Material"]
titanium=df.at[4, " Material"]

print(steel)
print(aluminum)
print(titanium)
df.drop(df.index, axis=0, inplace=True)
# df.columns


 AISI 4130 Steel#^# annealed at 865C@SOLIDWORKS Materials$SOLIDWORKS Materials|AISI 4130 Steel#^# annealed at 865C|94$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|AISI 4130 Steel#^# annealed at 865C|94
 6061-T6 (SS)@SOLIDWORKS Materials$SOLIDWORKS Materials|6061-T6 (SS)|164$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|6061-T6 (SS)|164
 Ti-6Al-4VSolution treated and aged (SS)@SOLIDWORKS Materials$SOLIDWORKS Materials|Ti-6Al-4VSolution treated and aged (SS)|205$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|Ti-6Al-4VSolution treated and aged (SS)|205


In [17]:
biked=pd.read_csv("BIKED_reduced.csv", index_col=0)
biked_rethick=PreSW.shufflethickness(biked)
# print(biked_rethick)

In [18]:
# count=0
bridges=[]
df=PreSW.renameCols(df, biked)
        
        
for idx in df.index:
#Set bridge diameter values based on whether bridges are on. If bridges are off, set diameter to 0.
        if biked.at[idx, "SEATSTAYbrdgCheck"]==1:
            df.at[idx, " SSB OD"]=biked.at[idx, "SEATSTAYbrdgdia1"]
            if biked.at[idx, "SEATSTAYbrdgdia1"]<0:
                invalid=True
        else:
            df.at[idx, " SSB OD"]=0
        if biked.at[idx, "CHAINSTAYbrdgCheck"]==1:
            df.at[idx, " CSB OD"]=biked.at[idx, "CHAINSTAYbrdgdia1"]
            if biked.at[idx, "CHAINSTAYbrdgdia1"]<0:
                invalid=True
        else:
            df.at[idx, " CSB OD"]=0
        
        #Set Material names
        if df.at[idx, " Material"]=="STEEL":
            df.at[idx, " Material"]=steel
        elif df.at[idx, " Material"]=="TITANIUM":
            df.at[idx, " Material"]=titanium
    #     if df.at[idx, " Material"]=="ALUMINIUM":
        else:
            df.at[idx, " Material"]=aluminum
        
# Code to count cases where bridges are wider than stays
#     if df.at[idx, " SSB OD"]>=df.at[idx, " SS OD"] and invalid==False:
#         bridges.append(idx)
#     elif df.at[idx, " CSB OD"]>=df.at[idx, " CS OD"] and invalid==False:
#         bridges.append(idx)


#Scale values from mm to meters, excluding any angles and material parameter        
columns=list(df.columns)
columns.remove(" Material")
columns.remove(" HT Angle")
columns.remove(" ST Angle")
scaleddf=df[columns].divide(1000)
scaleddf[" Material"]=df[" Material"]
scaleddf[" HT Angle"]=df[" HT Angle"]
scaleddf[" ST Angle"]=df[" ST Angle"]
scaleddf=scaleddf[df.columns]

#Save the dataframe before dropping invalid bikes for future use in validity classification
scaleddf.to_csv("biked_predrop.csv")

invalid_bikes, perc=getInvalid.getInvalid(scaleddf)
print(invalid_bikes)
scaleddf.drop(invalid_bikes, inplace=True)

# print(np.array(bridges))
# np.save("./Batch Info/bridges", np.array(bridges))

[3, 6, 11, 17, 20, 23, 30, 31, 57, 68, 109, 110, 122, 158, 177, 188, 189, 195, 199, 234, 240, 248, 253, 289, 290, 325, 329, 332, 336, 345, 353, 359, 372, 390, 391, 398, 403, 410, 429, 432, 442, 453, 455, 503, 505, 516, 525, 528, 542, 553, 579, 667, 671, 713, 749, 771, 844, 858, 918, 921, 947, 987, 990, 1013, 1040, 1042, 1048, 1062, 1065, 1110, 1151, 1154, 1169, 1183, 1196, 1218, 1232, 1233, 1253, 1284, 1299, 1333, 1336, 1344, 1345, 1346, 1347, 1355, 1356, 1366, 1368, 1369, 1377, 1382, 1387, 1407, 1413, 1416, 1434, 1435, 1453, 1464, 1472, 1490, 1496, 1555, 1556, 1640, 1669, 1693, 1700, 1736, 1761, 1765, 1771, 1783, 1787, 1789, 1790, 1791, 1793, 1801, 1805, 1810, 1823, 1829, 1863, 1873, 1891, 1908, 1926, 1956, 1957, 1962, 1988, 2016, 2036, 2075, 2110, 2148, 2162, 2192, 2259, 2260, 2268, 2313, 2326, 2405, 2422, 2428, 2435, 2472, 2482, 2518, 2596, 2606, 2641, 2643, 2674, 2675, 2676, 2691, 2756, 2808, 2838, 2839, 2866, 2881, 2883, 2930, 2931, 2939, 2968, 2995, 3018, 3032, 3095, 3102, 3117, 

In [19]:
scaleddf.to_csv("Processed_structural_data.csv")

In [20]:
augdf=pd.read_csv("Processed_structural_data.csv", index_col=0)

# augdf=augdf[1200:1400]
# augdf=augdf.iloc[[1,20,64]]

# augdf=pd.read_csv("Processed_structural_data_repaired.csv", index_col=0)
# resim=np.load("./Batch Info/resim.npy")
# print(len(resim))
# augdf=augdf.loc[resim[1200:1400]]

# augdf=pd.read_csv("./Batch Info/Processed_structural_data_matfix.csv", index_col=0)
# augdf=augdf.iloc[[1,20,64]]
augdf=augdf.iloc[1600:1800]

In [22]:
for idx in augdf.index:
    if augdf.at[idx, " CS F"]==augdf.at[idx, " BB Length"]/2:
        augdf.at[idx, " CS F"]=augdf.at[idx, " CS F"]-.0001
        print("Adjusting CS F for model " + str(idx) + " by 0.0001 since it is half of BB length and would cause model to lock!")


template=pd.read_csv("Design Study Template.csv", encoding="ISO-8859-1")
# if "index" in template.columns:
#     template.drop(["index"], axis=1, inplace=True)
newindex=augdf.columns.insert(0,"Status")
augdf["Status"]=np.full(len(augdf.index), 1)
augdf=augdf[newindex]
augdf.index=["Bike" + str(i) for i in augdf.index]
# print(template)
template.reset_index(inplace=True)  
units=template.iloc[1]

#Old code to drop "Level_0", replace this by dropping entries from top of units until reaching correct length
#     units.drop(["level_0"], axis=0, inplace=True)

while len(units.index)>len(augdf.columns):
    units.drop([units.index[0]], axis=0, inplace=True)
units.index=augdf.columns
units.rename("Units", inplace=True)
augdf=augdf.append(units)
newindices= augdf.index[:-1].insert(0, augdf.index[-1])
augdf=augdf.reindex(newindices)
augdf.columns.rename("Parameters", inplace=True)

augdf=augdf.T.reset_index().T
cols=[len(augdf.index)-2, len(augdf.columns)-1] + [""]*(len(augdf.columns)-2)
# print(cols)
augdf.columns=cols
augdf.columns.rename("Design Set", inplace=True)
# print(units)
print(augdf)

Design Set     200          35                                              \
Parameters  Status   CS Length   BB Drop   Stack   SS E   ST Angle   BB OD   
Units                        m         m       m      m        deg       m   
Bike1720         1       0.405     0.075  0.5656   0.06         73    0.04   
Bike1721         1       0.381    -0.007  0.5656  0.045         70    0.04   
Bike1722         1       0.381    -0.007  0.5656  0.045         70    0.04   
...            ...         ...       ...     ...    ...        ...     ...   
Bike1936         1        0.37     -0.01  0.5656   0.04         74    0.04   
Bike1937         1        0.39     0.034  0.5656  0.045         71    0.04   
Bike1938         1        0.39     0.005  0.5656  0.045         71    0.04   
Bike1939         1         0.4         0  0.5656  0.039         74    0.04   
Bike1940         1         0.4      0.03  0.5656  0.039       71.5    0.04   

Design Set                          ...                        

In [23]:
augdf.to_csv("generated_param_study_template.csv", index_label="Design Set")